<a href="https://colab.research.google.com/github/JYAnchang/LEAFAREAINDEX_PARTITIONING/blob/main/LAI_PARTITIONING_VERTICAL_SLICE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import modules

In [21]:
pip install geemap

In [22]:
import ee
import geemap


In [25]:
Map = geemap.Map(center=(15, -14), zoom=8)
Map

Map(center=[15, -14], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [ ]:
# Initialize the Earth Engine module.
ee.Initialize()

In [26]:
# Import data
modis_lai = ee.ImageCollection("MODIS/061/MCD15A3H")
wcc = ee.Image("projects/ee-slabnmsu-servir-assets/assets/east_africa_canopycover_2020")
prec = ee.Image("projects/ee-slabnmsu-servir-assets/assets/MAP_GLOBAL_1991_2020")

Create visualization parameters and define area of interest

In [27]:
# LAI visualization parameters
laiVisParams = {'min': 1, 'max': 6.4, 'palette': ["ff5b35", "ffdb28", "168b0e", "437dff"]}

# Define area (use footprint of woody cover data)
aoi = wcc.geometry()

Map.centerObject(aoi, 6)


specific sites to test LAI partitioning algorithm

In [28]:
# Locations to test timeseries, differentiated by amount of woody cover
grassland = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([37.295, 0.256]))])
shrubland = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([37.34, 0.297]))])
forest = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([37.37, 0.15]))])

Resample Woody cover (%) and 30 year Mean annual rainfall data to MODIS LAI scale

In [29]:
prj = modis_lai.first().select('Lai').projection()
wcc500 = wcc.clip(aoi).divide(100).resample('bilinear').reproject(prj).select([0], ['wcc'])
MeanAP = prec.clip(aoi).resample('bilinear').reproject(prj)

Define dates for filtering 4-day MODIS LAI data for the last 2 years

In [34]:
latest = ee.Date(modis_lai.sort('system:time_start', False).first().get('system:time_start'))
currentYear = ee.Number.parse(latest.format('YYYY'))
prevYear = currentYear.subtract(1)

raw = modis_lai.filter(ee.Filter.calendarRange(prevYear, currentYear,'year'));

Filter LAI data using quality flags and Clip to area of interest

In [35]:
# Functions for bitwise quality data masking of MODIS LAI
def bitwiseExtract(input, fromBit, toBit):
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return input.rightShift(fromBit).bitwiseAnd(mask)

def qFilter(image):
    image = image.clip(aoi)
    qc = image.select('FparLai_QC')
    qaMask = bitwiseExtract(qc, 0, 0).eq(0)
    cloudMask = bitwiseExtract(qc, 3, 4).eq(0)
    scfMask = bitwiseExtract(qc, 5, 7).lte(1)
    mask = qaMask.and(cloudMask).and(scfMask)
    return image.select('Lai').multiply(0.1).updateMask(mask).copyProperties(image, ['system:time_start'])

qFiltered = raw.map(qFilter)


SyntaxError: invalid syntax (<ipython-input-35-5c3ea3ecb974>, line 13)

The following functions and steps will be used to smooth the LAI data. This is a critical step.

In [ ]:
# Create list of dates from filtered collection metadata
n = ee.Number(raw.size())
indexList = ee.List.sequence(0, n.subtract(1), 2)
dateList = indexList.map(lambda i: ee.Date(ee.Image(qFiltered.toList(n).get(i)).get('system:time_start')))

# Interpolation for gap filling
def linearInterpolate(imageCollection, interval):
    interpolate = require('users/savannalabnmsu/misc:interpolateCol.js')
    return interpolate.linearInterpolate(qFiltered,32)

# remove low LAI values using 75th percentile
def upperProfileMean(coll, date, window, quantile):
    n = ee.Number(window)
    sub = coll.filterDate(date.advance(n.multiply(-1), 'day'), date.advance(n.add(1), 'day')).select('Lai')
    qImage = sub.reduce(ee.Reducer.percentile([quantile])).select(0)
    return sub.map(lambda img: img.updateMask(img.gte(qImage)).toFloat()).mean().set('system:time_start', date.millis()).set('date', date.format('YYYY-MM-dd'))

# moving window average for final pass smoothing (can also experiment with other smoothers like savitzy golay. This runs faster)
def movingMean(coll, date, window):
    n = ee.Number(window)
    return coll.filterDate(date.advance(n.multiply(-1), 'day'), date.advance(n.add(1), 'day')).select('Lai').mean().toFloat().set('system:time_start', date.millis()).set('date', date.format('YYYY-MM-dd'))

# apply functions
gapFilled = linearInterpolate(qFiltered, 32)
noLowVals = ee.ImageCollection.fromImages(dateList.map(lambda d: upperProfileMean(gapFilled, ee.Date(d), 16, 75)))
smoothCol = ee.ImageCollection.fromImages(dateList.map(lambda d: movingMean(noLowVals, ee.Date(d), 16)))

test smoothing

In [ ]:
print(ui.Chart.image.series(raw, grassland, ee.Reducer.mean(), 500))
print(ui.Chart.image.series(smooth, grassland, ee.Reducer.mean(), 500))